## Notbook that plots a few (fancy) maps in 

In [1]:
from netCDF4 import Dataset as nc, num2date
import numpy as np
import pandas as pd
import sys, os
import matplotlib
from matplotlib import pyplot as plt, cm
from mpl_toolkits.basemap import Basemap, cm as cm_b
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

Define all filenames

In [2]:
cpolF = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_1998-2017.nc')
cmorphF = os.path.join(os.environ['HOME'], 'Data', 'Darwin', 'netcdf', 'Cmorph_1999-2003.nc')
outDir = os.path.join(os.environ['HOME'], 'Data','Extremes' ,'CPOL', 'Plot')
DFFile = os.path.join(os.environ['HOME'],'Data','Extremes','CPOL','CPOL_MonsoonPhases-dates.hdf5')

Define the dates for plotting

In [4]:
periods = {}
keys = []
with pd.HDFStore(DFFile,'r') as h5:
    for key in h5.keys():
        keys.append(key[1:])
for key in keys:
    periods[key] = pd.read_hdf(DFFile,key)
periods['breaks'].start

#start, end = ('1999-01-01','1999-01-29'), ('1999-01-02','1999-02-01')
#T =[]
#for s, e in zip(start, end):
#    T += list(pd.date_range(s, e, freq='10min').to_pydatetime())
#T = pd.DatetimeIndex(T)

0     1998-12-06
1     1998-12-12
2     1998-12-14
3     1998-12-22
4     1998-12-30
5     1999-01-04
6     1999-01-14
7     1999-02-02
8     1999-02-12
9     1999-02-15
10    1999-03-09
11    1999-03-28
12    1999-04-28
13    1999-11-04
14    1999-11-18
15    1999-12-18
16    2000-01-14
17    2000-02-08
18    2000-03-03
19    2000-03-13
20    2001-11-07
21    2001-11-27
22    2001-12-09
23    2002-01-03
24    2002-01-10
25    2002-02-05
26    2002-02-10
27    2002-02-15
28    2002-11-05
29    2002-11-15
         ...    
41    2004-02-23
42    2004-03-16
43    2004-11-08
44    2004-11-22
45    2004-12-09
46    2004-12-24
47    2005-01-22
48    2005-02-03
49    2005-03-16
50    2005-11-10
51    2005-11-18
52    2005-12-08
53    2005-12-26
54    2006-01-12
55    2006-03-06
56    2006-04-18
57    2006-10-26
58    2006-11-02
59    2006-12-07
60    2007-01-08
61    2007-01-20
62    2007-01-28
63    2007-02-28
64    2007-03-09
65    2010-03-28
66    2010-11-13
67    2010-11-15
68    2010-12-

Get all timesteps

### Plot the CPOL vs CMORPH map
Create the maps first, for this get the metadata

In [111]:
with nc(cmorphF) as fnc:
    lon_cm = fnc.variables['lon'][:]
    lat_cm = fnc.variables['lat'][:]
    t_cm = pd.DatetimeIndex(num2date(fnc.variables['time'][:], fnc.variables['time'].units))
    with nc(cpolF) as gnc:
        lon_cp = gnc.variables['lon'][:]
        lat_cp = gnc.variables['lat'][:]
        cpol_times ={}
        for tt in ('10min', '1h', '3h', '6h'):
            cpol_times[tt] = pd.DatetimeIndex(num2date(gnc[tt].variables['time'][:], gnc[tt].variables['time'].units))
        #tt = np.array([(np.where(t_cm == p)[0][0], np.where(t_cp == p)[0][0])for p in T if p in t_cp and p in t_cm])
        idx = []
        for tt in T:
            if tt in cpol_times['10min']:
                try:
                    i10m=np.where(cpol_times['10min'] == tt)[0][0]
                    t1h = pd.Timestamp(tt.year, tt.month, tt.day, tt.hour)
                    i1h = np.where(cpol_times['1h'] == t1h)[0][0]
                    c = np.array(3*list(range(int(24/3))));c.sort();c*=3
                    i3h = np.where(cpol_times['3h'] == pd.Timestamp(tt.year, tt.month, tt.day, c[tt.hour]))[0][0]
                    i3hc = np.where(t_cm == pd.Timestamp(tt.year, tt.month, tt.day, c[tt.hour]))[0][0]
                    c = np.array(6*list(range(int(24/6))));c*=6;c.sort()
                    i6h = np.where(cpol_times['6h'] == pd.Timestamp(tt.year, tt.month, tt.day, c[tt.hour]))[0][0]
                    idx.append((i10m,i1h,i3h,i6h,i3hc))
                except IndexError:
                    pass
        
        idx = np.array(idx)
        rr_cp = {}
        for n,tt in enumerate(('10min', '1h', '3h', '6h')):
            rr_cp[tt] = gnc[tt].variables['rain_rate'][idx[:,n]]
    rr_cm = fnc.variables['precip'][idx[:,-1]]

In [155]:
m_cm = Basemap(llcrnrlat=min(lat_cm), llcrnrlon=min(lon_cm), urcrnrlat=max(lat_cm), urcrnrlon=max(lon_cm), resolution='i')
matplotlib.rcParams['figure.figsize'] = [30,30]
avg_type = {'10min':'(10 Min.)', '1h': '(1 hly)', '3h': '(3 hly)', '6h': '(6 hly)'}

In [135]:
len()

578

In [158]:
first = True
for i in range(44,len(rr_cm)):
    fig = plt.figure(figsize=(20,15), dpi=72)
    ax1 = plt.subplot2grid((3,3), (0,1))
    ax10 = plt.subplot2grid((3,3), (0,0))
    ax6 = plt.subplot2grid((3,3), (0,2))
    ax = plt.subplot2grid((3,3), (1,0), rowspan=2, colspan=3)
    outer_title = 'CMORPH rain-rates at %s' %t_cm[idx[i,-1]].strftime('%Y-%m-%d %H:%M')
    inner_title = 'Cpol rain-rate at %s' %cpol_times['10min'][idx[i,1]].strftime('%Y-%m-%d %H:%M')
    try:
        rr_out = rr_cm[i].filled(0) / 3. /2
    except AttributeError:
        rr_out = rr_cm[i] / 3. /2       
    im = m_cm.pcolormesh(lon_cm, lat_cm, rr_out, vmin=0.5, vmax=5., cmap='Blues')
    m_cm.drawcoastlines()
    cbar=m_cm.colorbar(im,location='bottom',pad='5%')
    cbar.set_label('Rain-rate mm/h',size=26)
    cbar.ax.tick_params(labelsize=26)
    axins = zoomed_inset_axes(ax, 6, loc=3)
    axins.set_xlim(min(lon_cp), max(lon_cp))
    axins.set_ylim(min(lat_cp), max(lon_cp))
    #if first:
    m_cp_i = Basemap(llcrnrlat=min(lat_cp), llcrnrlon=min(lon_cp), urcrnrlat=max(lat_cp), urcrnrlon=max(lon_cp), ax=axins, resolution='i')
    m_cp_1 = Basemap(llcrnrlat=min(lat_cp), llcrnrlon=min(lon_cp), urcrnrlat=max(lat_cp), urcrnrlon=max(lon_cp), ax=ax1, resolution='i')
    m_cp_10 = Basemap(llcrnrlat=min(lat_cp), llcrnrlon=min(lon_cp), urcrnrlat=max(lat_cp), urcrnrlon=max(lon_cp), ax=ax10, resolution='i')
    m_cp_6 = Basemap(llcrnrlat=min(lat_cp), llcrnrlon=min(lon_cp), urcrnrlat=max(lat_cp), urcrnrlon=max(lon_cp), ax=ax6, resolution='i')
    first = False
    plt.xticks(visible=False)
    plt.yticks(visible=False)
    for m, name, a, ii in ((m_cp_10,'10min', ax10, 0),(m_cp_1,'1h', ax1, 1),(m_cp_i,'3h', axins, 2),(m_cp_6,'6h', ax6, 3)):
        m.pcolormesh(lon_cp, lat_cp, rr_cp[name][i], vmin=0.5, vmax=5. ,cmap='Blues')
        m.drawcoastlines()
        if name != '3h':
            a.set_title('Cpol at %s %s'%(cpol_times[name][idx[i,ii]].strftime('%Y-%m-%d %H:%M'), avg_type[name]), size=18)
    mark_inset(ax, axins, loc1=4, loc2=1, fc="none", ec="0.5")
    ax.set_title(outer_title,size=26)
    plt.tight_layout()
    fname=os.path.join(outDir,'comparison-%03i.png'%i)
    fig.savefig(fname, dpi=72, facecolor='w', edgecolor='w', bbox_inches='tight')
    #plt.show()
    ax1.cla(),ax10,ax.cla(),ax6.cla(), fig.clf(), plt.close()
    #break